In [58]:
import json
import binascii
import os
import urllib3
import certifi
import requests
from datetime import date, datetime, timedelta
import time

In [ ]:
# Get authorisation params
with open('user_token_params.json', 'r', encoding='utf-8') as f:
    user_token_params = json.load(f)

In [52]:
vacancy_id = '70867905' # ИТ Директор (Солнечногорск)
resume_id = '6e1019f2ff040603530039ed1f4b59356a4b71' # IT
message = ''
response_params = {}

In [ ]:
#----------------------------------------------------------------------------
# https://julien.danjou.info/handling-multipart-form-data-python/
#----------------------------------------------------------------------------
def encode_multipart_formdata(fields):

    boundary = binascii.hexlify(os.urandom(16)).decode('ascii')
    body = (
        "".join("--%s\r\n"
                "Content-Disposition: form-data; name=\"%s\"\r\n"
                "\r\n"
                "%s\r\n" % (boundary, field, value)
                for field, value in fields.items()) +
        "--%s--\r\n" % boundary
    )
    content_type = "multipart/form-data; boundary=%s" % boundary
    return body, content_type

In [26]:
#------------------------------------------------------------------------------
#
#------------------------------------------------------------------------------
http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED', ca_certs=certifi.where())

In [ ]:
body, content_type = encode_multipart_formdata({'vacancy_id': '{}'.format(vacancy_id),
                           'resume_id' : '{}'.format(resume_id),
                           'message'   : '{}'.format(message)})

r = http.request('POST', 'https://api.hh.ru/negotiations',
                                    headers = {'Authorization' : 'Bearer {}'.format(user_token_params['access_token']),
                                              'Accept'        : '*/*',
                                              'User-Agent'    : 'CIO_jbSearch (ssv.ruby@gmail.com)',
                                              'Content-Type'  : '{}'.format(content_type)},
                                    body =  '{}'.format(body)
                                  )

In [55]:
response_params['status'] = r.status
response_params['reason'] = r.reason
response_params['date'] = r.getheader('Date')
response_params['response_link'] = 'https://hh.ru/applicant{}'.format(r.getheader('Location'))
response_params

{'status': 201,
 'reason': 'Created',
 'date': 'Sat, 08 Oct 2022 14:03:16 GMT',
 'response_link': 'https://hh.ru/applicant/negotiations/2928346342'}

## -------------------------------------------------------------------------
1. Получить список поисковых запросов (`get_search_url()`)
2. В цикле (с интервалом N-минут):
2.1 Сформировать список новых вакансий (`get_vacancies()`)
2.2 Сформировать и отправить отклик

In [105]:
#----------------------------------------------------------------------
# Чтение из файла search_url_list.txt поисковых запросов по сайту
# ----------------------------------------------------------------------
def get_search_url(search_url_file = 'search_url_list.txt'):

    search_url_list = []

    with open(search_url_file, 'r') as f:
        urls = f.readlines()

    for current_string in urls:
        if current_string[0:1] == '#' or current_string[0:1] == '\n':
            continue
        search_url_list.append(current_string.rstrip())

    return search_url_list


In [66]:
def get_vacancies(url_list, vacancies, log_file_name='vacancy_log.txt'):

    date_from = (date.today() - timedelta(hours=24)).isoformat()
    searches = []
    new_vacancies = []

    with open(log_file_name, 'a') as logfile:

        for url in url_list:
            for page in range(20):
                try:
                    searches.append(requests.get(url + '&page={}&per_page=100&date_from={}'.format(page, date_from)).json())
                except Exception as ex_err:
                    logfile.write('==> ERROR: {} W= Read Vacansies ===> {}'.format(date.today().isoformat(), ex_err) + '\n')

        try:
            for items in searches:
                for j in items['items']:
                    if j['id'] not in vacancies:
                        vacancies.append(j['id'])
                        new_vacancies.append(j['id'])
        except:
            pass

    return new_vacancies

In [67]:
# ----------------------------------------------------------------------
#  Список просмотренных вакансий (Глобальная переменная)
# ----------------------------------------------------------------------
vacancies = []

In [109]:
url_list = get_search_url()
get_vacancies(url_list, vacancies)

['70688374', '70681189']

In [110]:

print(vacancies)


['69698797', '69807951', '70205848', '69715592', '70670739', '70667911', '70066898', '69990798', '70050557', '70821609', '67553546', '70366313', '70817455', '70030830', '70202353', '69793288', '70688374', '70681189']
